# Lab3 - TLS - Lukas Forst
forstluk@fel.cvut.cz

## Task 1: Diffie–Hellman key exchange
Implement the [vanilla](https://en.wikipedia.org/wiki/Diffie–Hellman_key_exchange#Cryptographic_explanation) DH algorithm.
Try it with ``p=37`` and `g=5`. Can you make it working with recommended values
``p=0xFFFFFFFFFFFFFFFFC90FDAA22168C234C4C6628B80DC1CD129024E088A67CC74020BBEA63B139B22514A08798E3404DDEF9519B3CD3A431B302B0A6DF25F14374FE1356D6D51C245E485B576625E7EC6F44C42E9A637ED6B0BFF5CB6F406B7EDEE386BFB5A899FA5AE9F24117C4B1FE649286651ECE45B3DC2007CB8A163BF0598DA48361C55D39A69163FA8FD24CF5F83655D23DCA3AD961C62F356208552BB9ED529077096966D670C354E4ABC9804F1746C08CA18217C32905E462E36CE3BE39E772C180E86039B2783A2EC07A28FB5C55DF06F4C52C9DE2BCBF6955817183995497CEA956AE515D2261898FA051015728E5A8AACAA68FFFFFFFFFFFFFFFF``
and ``g=2`` ?

In [6]:
DEFAULT_P = 0xFFFFFFFFFFFFFFFFC90FDAA22168C234C4C6628B80DC1CD129024E088A67CC74020BBEA63B139B22514A08798E3404DDEF9519B3CD3A431B302B0A6DF25F14374FE1356D6D51C245E485B576625E7EC6F44C42E9A637ED6B0BFF5CB6F406B7EDEE386BFB5A899FA5AE9F24117C4B1FE649286651ECE45B3DC2007CB8A163BF0598DA48361C55D39A69163FA8FD24CF5F83655D23DCA3AD961C62F356208552BB9ED529077096966D670C354E4ABC9804F1746C08CA18217C32905E462E36CE3BE39E772C180E86039B2783A2EC07A28FB5C55DF06F4C52C9DE2BCBF6955817183995497CEA956AE515D2261898FA051015728E5A8AACAA68FFFFFFFFFFFFFFFF
DEFAULT_G = 2
DEFAULT_BLOCK_SIZE = 16

In [7]:
import hashlib
import secrets

class DhAgent:

    def __init__(self, msg: str = None, **kwargs):
        self.msg = msg
        self.p = kwargs.pop('p', DEFAULT_P)
        self.g = kwargs.pop('g', DEFAULT_G)
        self.block_size = kwargs.pop('block_size', DEFAULT_BLOCK_SIZE)

        self.private_key = secrets.randbelow(self.p)

        self.shared_secret = None

    def send_public_data(self) -> [int, int]:
        return self.p, self.g

    def receive_public_data(self, p: int, g: int):
        self.p, self.g = p, g

    def send_public_key(self) -> int:
        return pow(self.g, self.private_key, self.p)

    def receive_public_key(self, pk: int):
        self.shared_secret = pow(pk, self.private_key, self.p)

    def key(self) -> bytes:
        # TODO check if it is necessary to get bytes of int or str(secret) is ok
        sha1 = hashlib.sha1(str(self.shared_secret).encode('utf-8')).digest()
        # TODO check if take first or last (imo it doesn't matter)
        return sha1[:self.block_size]

Now let's test the agent.

In [8]:
def assert_dh(p: int, g: int):
    alice = DhAgent(p=p, g=g)
    bob = DhAgent(p=p, g=g)

    alice.receive_public_key(bob.send_public_key())
    bob.receive_public_key(alice.send_public_key())

    assert alice.shared_secret == bob.shared_secret, "shared secrets don't match!"
    print("Ok")


p, g = 37, 5
assert_dh(p, g)

p = 0xFFFFFFFFFFFFFFFFC90FDAA22168C234C4C6628B80DC1CD129024E088A67CC74020BBEA63B139B22514A08798E3404DDEF9519B3CD3A431B302B0A6DF25F14374FE1356D6D51C245E485B576625E7EC6F44C42E9A637ED6B0BFF5CB6F406B7EDEE386BFB5A899FA5AE9F24117C4B1FE649286651ECE45B3DC2007CB8A163BF0598DA48361C55D39A69163FA8FD24CF5F83655D23DCA3AD961C62F356208552BB9ED529077096966D670C354E4ABC9804F1746C08CA18217C32905E462E36CE3BE39E772C180E86039B2783A2EC07A28FB5C55DF06F4C52C9DE2BCBF6955817183995497CEA956AE515D2261898FA051015728E5A8AACAA68FFFFFFFFFFFFFFFF
g = 2
assert_dh(p, g)

Ok
Ok


## Task 2: Diffie–Hellman key
Turn a DH secret into a key. Use ``sha1`` to generate `BLOCK_SIZE = 16` long key material.

In [9]:
def alice_bob() -> [DhAgent, DhAgent]:
    # create agents
    alice = DhAgent()
    bob = DhAgent()
    # exchange key material
    alice.receive_public_key(bob.send_public_key())
    bob.receive_public_key(alice.send_public_key())
    # return instances
    return alice, bob

Now, let's test the implementation

In [10]:
alice, bob = alice_bob()

assert alice.key() == bob.key()
assert len(alice.key()) == DEFAULT_BLOCK_SIZE

print("Ok")

Ok


## Task 3: Bulk cipher
Ensure you have working implementation of AES in CBC mode with PKCS&#35;7 padding. It is recommended to use  `BLOCK_SIZE = 16`
You will need ``encrypt(key, iv, message)`` and `decrypt(key, iv, encrypted_message)` functions.
You can check your implementation with ``bulk_cipher.py`` example.


In [11]:
from Cryptodome.Cipher import AES
from pkcs7 import PKCS7Encoder

def encrypt(key: bytes, iv: bytes, message: str):
    encoder = PKCS7Encoder(DEFAULT_BLOCK_SIZE)

    aes = AES.new(key, AES.MODE_CBC, iv[:16])
    pad_text = encoder.encode(message)
    return aes.encrypt(pad_text.encode())


def decrypt(key: bytes, iv: bytes, encrypted_message: bytes) -> str:
    encoder = PKCS7Encoder(DEFAULT_BLOCK_SIZE)
    
    aes = AES.new(key, AES.MODE_CBC, iv[:16])
    pad_text = aes.decrypt(encrypted_message)
    return encoder.decode(pad_text.decode())

Now, let's test the implementation.

In [13]:
import random
import string
import os

BLOCK_SIZE = DEFAULT_BLOCK_SIZE

key = os.urandom(BLOCK_SIZE)
iv = os.urandom(BLOCK_SIZE)
msg = ''.join(random.choice(string.ascii_lowercase) for i in range(1024))

ciphertext = encrypt(key, iv, msg)
cleartext = decrypt(key, iv, ciphertext)

assert cleartext == msg
print("Ok")

Ok


## Task 4: Implement simple SSL/TLS setup
It's time to have some fun now. Checkout `tls_101.py` example. Implement `Agent()` class such that this code executes with no errors.
You might want to use DH keys to seed AES_CBC bulk cipher you have implemented before
The interface for the ``Agent()`` class should support:
* sending/receiving public data (`p` and `g`)
* sending/receiving public key
* sending/receiving messages

Please, use recommended values for `p` and `g` for DH key exchange protocol.

In [14]:
class Agent(DhAgent):
    def send_message(self) -> bytes:
        iv = os.urandom(self.block_size)
        cipher_text = encrypt(self.key(), iv, self.msg)
        return iv + cipher_text

    def receive_message(self, msg: bytes):
        iv = msg[0:self.block_size]
        cipher_text = msg[self.block_size:]
        self.msg = decrypt(self.key(), iv, cipher_text)

Now let's test Agent.

In [15]:
alice = Agent("I'M 5UppER Kewl h4zKEr")
bob = Agent()


# Alice has da message, Bob doesn't
assert alice.msg
assert not bob.msg

# Negotiate parameters publicly
bob.receive_public_data(*alice.send_public_data())
alice.receive_public_data(*bob.send_public_data())

# Exchange keys publicly
bob.receive_public_key(alice.send_public_key())
alice.receive_public_key(bob.send_public_key())

# Pass da message
ciphertext = alice.send_message()
bob.receive_message(ciphertext)
# Bob has it now
assert alice.msg == bob.msg
print("Ok")

Ok


## Task 5: Man-in-the-middle
Oh, no! Looks like something is wrong here! Who the hell is Mallory?
Implement `MITM()` class such that `itls_101.py` runs with no errors.
The interface should support:
* sending/receiving public data (`p` and `g`)
* sending/receiving public key
* intercept_message

In [16]:
class MITM():
    def __init__(self):
        self.alice_proxy = Agent()
        self.bob_proxy = Agent()
        
        # get same private key for all instances
        self.bob_proxy.private_key = self.alice_proxy.private_key
        
        self.received_pd = 0
        self.received_pk = 0
    
    def send_public_data(self) -> [int, int]:
        return self.bob_proxy.send_public_data()

    def receive_public_data(self, p: int, g: int):
        if self.received_pd == 0:
            self.alice_proxy.receive_public_data(p, g)
        elif self.received_pd == 1:
            self.bob_proxy.receive_public_data(p, g)

        self.received_pd = self.received_pd + 1

    def send_public_key(self) -> int:
        return self.bob_proxy.send_public_key()

    def receive_public_key(self, pk: int):
        if self.received_pk == 0:
            self.alice_proxy.receive_public_key(pk)
        elif self.received_pk == 1:
            self.bob_proxy.receive_public_key(pk)

        self.received_pk = self.received_pk + 1

    def intercept_message(self, alice_cipher_text: bytes) -> bytes:
        # decrypt from alice
        self.alice_proxy.receive_message(alice_cipher_text)
        self.msg = self.alice_proxy.msg
        # encrypt for bob
        self.bob_proxy.msg = self.msg
        return self.bob_proxy.send_message()

Now let's test the code.

In [17]:
alice = Agent("I'M 5UppER Kewl h4zKEr")
bob = Agent()
mallory = MITM()

# Alice has da message, Bob doesn't
assert alice.msg
assert not bob.msg

# Negotiate parameters publicly
mallory.receive_public_data(*alice.send_public_data())
bob.receive_public_data(*mallory.send_public_data())
mallory.receive_public_data(*bob.send_public_data())
alice.receive_public_data(*mallory.send_public_data())

# Exchange keys publicly
mallory.receive_public_key(alice.send_public_key())
bob.receive_public_key(mallory.send_public_key())
mallory.receive_public_key(bob.send_public_key())
alice.receive_public_key(mallory.send_public_key())

# Pass da message
bob.receive_message(mallory.intercept_message(alice.send_message()))
# Bob has it now
assert bob.msg == alice.msg
# Mallory too
assert mallory.msg == alice.msg
print("Ok")

Ok


## Task 6: RSA
[RSA algorithm](https://en.wikipedia.org/wiki/RSA_(cryptosystem)#Key_generation) is the most used asymmetric encryption algorithm in the world. It is based on the principal that it is easy to multiply large numbers, but factoring large numbers is very hard.
Within the TLS context it is used for both key exchange and generate signatures for security certificates (do you know why is that possible?). Let us implement this algorithm.
Here are few hints:
* Please use `p = 13604067676942311473880378997445560402287533018336255431768131877166265134668090936142489291434933287603794968158158703560092550835351613469384724860663783`, `q = 20711176938531842977036011179660439609300527493811127966259264079533873844612186164429520631818559067891139294434808806132282696875534951083307822997248459` and `e=3` for the key generation procedure.
* You might want to implement your `invmod` function. Test it with values `a=19` and `m=1212393831`. You should get `701912218`.
Your function should also correctly handles the case  when `a=13` and `m=91`
* You might want to implement functions `encrypt(bytes_, ...)/decrypt(bytes_,...)` and separately `encrypt_int(int_, ...)/decrypt_int(int_,...)`
* Please use [big endian](https://en.wikipedia.org/wiki/Endianness#Big-endian) notation when transforming bytes to integer

In [18]:
DEFAULT_P = 13604067676942311473880378997445560402287533018336255431768131877166265134668090936142489291434933287603794968158158703560092550835351613469384724860663783
DEFAULT_Q = 20711176938531842977036011179660439609300527493811127966259264079533873844612186164429520631818559067891139294434808806132282696875534951083307822997248459
DEFAULT_E = 3

In [19]:
# in latest python, invmod is easy
def invmod(a, m):
    try:
        return pow(a, -1, m)
    except:
        return 1
assert invmod(19, 1212393831) == 701912218

# https://stackoverflow.com/a/30375198/7169288
def int_to_bytes(x: int) -> bytes:
    return x.to_bytes((x.bit_length() + 7) // 8, 'big')
    
def int_from_bytes(xbytes: bytes) -> int:
    return int.from_bytes(xbytes, 'big')

In [20]:
import math
from dataclasses import dataclass

@dataclass
class PrivateKey:
    n: int
    d: int

@dataclass
class PublicKey:
    n: int
    e: int
        
def generate_key(**kwargs):
    p = kwargs.pop('p', DEFAULT_P)
    q = kwargs.pop('q', DEFAULT_Q)
    e = kwargs.pop('e', DEFAULT_E)
    
    n = p*q
    t = math.lcm(p - 1, q - 1)
    d = invmod(e, t)
    return PrivateKey(n, d), PublicKey(n, e)

def encrypt_int(public_key: PublicKey, message: int) -> int:
    return pow(message, public_key.e, public_key.n)

def decrypt_int(private_key: PrivateKey, cipher_text: int) -> int:
    return pow(cipher_text, private_key.d, private_key.n)

def encrypt(public_key: PublicKey, message: bytes) -> bytes:
    c = encrypt_int(public_key, int_from_bytes(message))
    return int_to_bytes(c)

def decrypt(private_key: PrivateKey, cipher_text: bytes) -> bytes:
    m = decrypt_int(private_key, int_from_bytes(cipher_text))
    return int_to_bytes(m)

Now, let's test the code.

In [21]:
private_key, public_key = generate_key(e=3)

message = "I will not write crypto code myself, but defer to high-level libraries written by experts who took the right decisions for me".encode()
cipher_text = encrypt(public_key, message)
assert message == decrypt(private_key, cipher_text)
print("Ok")

Ok


## Task 7:  RSA broadcast attack
It's time to check now that despite a really complex math involved in RSA algorithm it is still might be vulnerable to a number of attacks.
In this exercise we will implement the RSA broadcast attack (a.k.a simplest form of [Håstad's broadcast attack](https://en.wikipedia.org/wiki/Coppersmith's_attack#Håstad's_broadcast_attack))
Assume yourself an attacker who was lucky enough to capture any 3 of the ciphertexts and their corresponding public keys.
Check out `message_captured`. You also know that those ciphers a related to the same message. Can you read the message? Here are a few hints for this exercise:
* The data is encrypted using `encrypt_int(public, bytes2int(message.encode()))`.
* Please note, that in all 3 case public keys _are different_

How Chinese remainder theorem is helping you here?

First, let's parse data.

In [22]:
import json

captured_ms = []
with open('message_captured', 'r') as f:
    captured_ms = [json.loads(m) for m in f.readlines()]

es = [m['e'] for m in captured_ms]

# check that all e are same
e = es[0]
assert es.count(e) == len(es)

ns = [m['n'] for m in captured_ms]
data = [m['data'] for m in captured_ms]

Now we use find Chinese reminder.

In [23]:
from functools import reduce

def chinese_remainder(ns: [int], cs: [int]) -> int:
    prod = reduce(lambda acc, b: acc*b, ns)
    s = 0
    for n, c in zip(ns, cs):
        p = prod // n
        s += c * invmod(p, n) * p
    return s % prod

# we can not do just math.pow(i,1/3.) as the ints are too big
# so we bruteforce it using bisection method
# https://stackoverflow.com/a/23622115/7169288
def find_cube_root(n: int) -> int:
    lo = 0
    hi = 1 << ((n.bit_length() + 2) // 3)
    while lo < hi:
        mid = (lo+hi)//2
        if mid**3 < n:
            lo = mid+1
        else:
            hi = mid
    return lo

Now we can find the plaintext.

In [24]:
chrm = chinese_remainder(ns, data)
cr = find_cube_root(chrm)

# aaaand we have a plaintext
int_to_bytes(cr)

b'Even experts make mistakes. Crypto is extremely difficult to get right'

## Task 8: Bleichenbacher's RSA attack
RSA is also used to generate digital signatures. When generating a signature the algorithm is somehow reversed: the message is first "decrypted" with a private key and then is being send over an open channel
to be "encrypted" with a public key known to a client. In this exercise we are going to implement the attack that broke Firefox's TLS certificate validation about 10 years years ago. The interested reader can refer to [this](https://mailarchive.ietf.org/arch/msg/openpgp/5rnE9ZRN1AokBVj3VqblGlP63QE) article.

The most widely used scheme for RSA signing at that was this: one takes the hash of the message to be signed, and then encodes it like this
```00 01 FF FF ... FF FF 00 ASN.1 HASH```.
Where ``ASN.1`` is a very complex binary encoding of the hash type and length.
The above then is  "decrypted" with RSA. `FF` bytes provide padding to make the message exactly as long as the modulus `n`.

The intuition behind the Bleichenbacher's RSA attack is that while it's impossible without private key (more specifically, without `d`) to
find a number that elevated to `e` gives exactly the encoding above, one can get to an approximation,
for example by taking the `e`-th root of the target message. If `e` is small enough, the approximation might be good enough to get a message like
``00 01 FF 00 ASN.1 HASH GARBAGE``


If the verification function fails to check that the hash is aligned at the end of the message (i.e. that there are enough `FF` bytes),
we can fake signatures that will work with any public key using a certain small `e`. As you can see, `n` becomes completely irrelevant
because exponentiation by `e` never wraps past the modulus.

In this exercise you will be asked to implement all the functions needed to make code ``rsa_bleichenbachers.py`` running without errors.
Please, use `p = 19480788016963928122154998009409704650199579180935803274714730386316184054417141690600073553930946636444075859515663914031205286780328040150640437671830139` and
`q = 17796969605776551869310475203125552045634696428993510870214166498382761292983903655073238902946874986503030958347986885039275191424502139015148025375449097`
for the key generation procedure. `e` as before is 3.

In [25]:
# see https://www.ietf.org/rfc/rfc3447.txt - page 42
ansi1 = b"\x30\x21\x30\x09\x06\x05\x2b\x0e\x03\x02\x1a\x05\x00\x04\x14"

def generate_message_hash(msg: bytes) -> bytes:
    return hashlib.sha1(msg).digest()

def rsa_signature_pad(n: int, msg_sha1: bytes) -> bytes:
    digest = b"\x00\x01"
    diff = (n.bit_length() // 8) - len(digest) - 1 - len(ansi1) - len(msg_sha1)
    digest = digest + b"\xff"* diff  + b"\x00" + ansi1 + msg_sha1
    return digest

def generate_signature(private_key: PrivateKey, msg_sha1: bytes) -> bytes:
    digest = rsa_signature_pad(private_key.n, msg_sha1)
    return decrypt(private_key, digest)

def verify_signature(public_key: PublicKey, signature: bytes, msg_sha1: bytes) -> bool:
    dec = encrypt(public_key, signature) # this returns without first zeros
    dec = dec if dec[0] == b"\x00" else b"\x00" + dec # put zero back to their place
    # check that it starts with correct pattern
    if dec[:2] != b"\x00\x01":
        return False
    
    # get rid of padding, find 00 separator before ANS.1
    # 00 01 FF.. 00 ASN.1 HASH ---->  len(ASN.1) + three bytes (00 01 00)
    hash_start_idx = dec[2:].find(b"\x00") + len(ansi1) + 3
    # select hash (it has length 20 because of sha1)
    signature_hash = dec[hash_start_idx:hash_start_idx + 20]
    return signature_hash == msg_sha1
    

def fake_signature(msg_sha1: bytes) -> bytes:
    # 128 is the lenght of signature, 15 ANS.1, 20 sha1 and 4 as padding bytes 00 01 FF 00
    trash_padding = 128 - len(ansi1) - len(msg_sha1) - 4
    digest = b"\x00\x01\xFF\x00" + ansi1 + msg_sha1 + (b"\x00" * trash_padding)
    return int_to_bytes(find_cube_root(int_from_bytes(digest)))

And now let's test it.

In [26]:
p = 19480788016963928122154998009409704650199579180935803274714730386316184054417141690600073553930946636444075859515663914031205286780328040150640437671830139
q = 17796969605776551869310475203125552045634696428993510870214166498382761292983903655073238902946874986503030958347986885039275191424502139015148025375449097
e = 3
message = b'Trust no one'
msg_sha1 = generate_message_hash(message)
private_key, public_key = generate_key(p=p, q=q, e=e)
signature = generate_signature(private_key, msg_sha1)
assert verify_signature(public_key, signature, msg_sha1)
assert verify_signature(public_key, fake_signature(msg_sha1), msg_sha1)

print("Ok")

Ok


## Task 9: DSA
The final task of this block is pretty simple. We are going to break Digital Signature Algorithm ([DSA](https://en.wikipedia.org/wiki/Digital_Signature_Algorithm)). If the used [nonce](https://en.wikipedia.org/wiki/Cryptographic_nonce)
is weak than it is trivial to break the DSA.

Let us set the DSA domain parameters as follows:

| Parameter        | Value           |
| ------------- |:-------------|
| p | `0x800000000000000089e1855218a0e7dac38136ffafa72eda7859f2171e25e65eac698c1702578b07dc2a1076da241c76c62d374d8389ea5aeffd3226a0530cc565f3bf6b50929139ebeac04f48c3c84afb796d61e5a4f9a8fda812ab59494232c7d2b4deb50aa18ee9e132bfa85ac4374d7f9091abc3d015efc871a584471bb1` |
| q | `0xf4f47f05794b256174bba6e9b396a7707e563c5b`      |
| g | `0x5958c9d3898b224b12672c0b98e06c60df923cb8bc999d119458fef538b8fa4046c8db53039db620c094c9fa077ef389b5322a559946a71903f990f1f7e0e025e2d7f7cf494aff1a0470f5b64c36b625a097f1651fe775323556fe00b3608c887892878480e99041be601a62166ca6894bdd41a7054ec89f756ba9fc95302291`      |

You also were lucky to capture the SHA1 of a message which is `H=0x2bc546792a7624fb6e972b0fb85081fd20a8a28`. Knowing my public key and DSA signature

| Parameter        | Value           |
| ------------- |:-------------|
| y | `0x33ff14f19fa9cf09b28747cdfe97252c4be46c9c4c2ee68a2231cb4b262dd839962eff659bd30f706e6cb2470117f211eadfadeac267bc4fecde6d4c058cdf5d7b8c75ba663ce7a87d22b171413b8d3b6ceee31b139051c385a06b8b2e2e587a15e87381c93f866bf7b122fda5c1f44d20480137906ed6026ed96c3793fde263` |
| r | `548099063082341131477253921760299949438196259240`      |
| s | `857042759984254168557880549501802188789837994940`      |

can you derive my private key? Its SHA-1 fingerprint (after being converted to hex) is: `0x8f96763dea794b79094eef4717ceb5f10631d634`. Implement your function `recover_private_key(dsa_params, dsa_sign, H, ...)` and send your code.

_Hint_: `k` is the number between `0` and `2**16`

In [27]:
def recover_private_key(dsa_params, dsa_sign, H, y) -> int:
    p, q, g = dsa_params
    r, s = dsa_sign
    inv_r = invmod(r, q)
    top = 2 ** 16
    for k in range(top):
        x = ((s * k - H) * inv_r) % q
        if pow(g, x, p) == y:
            print(f"Found! {x}")
            return x
        if k % 1000 == 0:
            print(f"{k}/{top}")

In [28]:
p = 0x800000000000000089E1855218A0E7DAC38136FFAFA72EDA7859F2171E25E65EAC698C1702578B07DC2A1076DA241C76C62D374D8389EA5AEFFD3226A0530CC565F3BF6B50929139EBEAC04F48C3C84AFB796D61E5A4F9A8FDA812AB59494232C7D2B4DEB50AA18EE9E132BFA85AC4374D7F9091ABC3D015EFC871A584471BB1
q = 0xF4F47F05794B256174BBA6E9B396A7707E563C5B
g = 0x5958C9D3898B224B12672C0B98E06C60DF923CB8BC999D119458FEF538B8FA4046C8DB53039DB620C094C9FA077EF389B5322A559946A71903F990F1F7E0E025E2D7F7CF494AFF1A0470F5B64C36B625A097F1651FE775323556FE00B3608C887892878480E99041BE601A62166CA6894BDD41A7054EC89F756BA9FC95302291

y = 0x33FF14F19FA9CF09B28747CDFE97252C4BE46C9C4C2EE68A2231CB4B262DD839962EFF659BD30F706E6CB2470117F211EADFADEAC267BC4FECDE6D4C058CDF5D7B8C75BA663CE7A87D22B171413B8D3B6CEEE31B139051C385A06B8B2E2E587A15E87381C93F866BF7B122FDA5C1F44D20480137906ED6026ED96C3793FDE263
r = 548099063082341131477253921760299949438196259240
s = 857042759984254168557880549501802188789837994940

H = 0x2BC546792A7624FB6E972B0FB85081FD20A8A28

recovered = recover_private_key([p, q, g], [r, s], H, y)

0/65536
1000/65536
2000/65536
3000/65536
4000/65536
5000/65536
6000/65536
7000/65536
8000/65536
9000/65536
10000/65536
11000/65536
12000/65536
13000/65536
14000/65536
15000/65536
16000/65536
Found! 40652980678670648677780565078388285335180708481


In [29]:
def check_fingerprint(expected: int, recovered_key: int) -> bool:
    recovered_hex_bytes = hex(recovered)[2:].encode()
    recovered_fingerprint = int_from_bytes(hashlib.sha1(recovered_hex_bytes).digest())
    return recovered_fingerprint == expected

real_fingerprint = 0x8f96763dea794b79094eef4717ceb5f10631d634
assert check_fingerprint(real_fingerprint, recovered)
print("Ok")

Ok
